<a href="https://colab.research.google.com/github/leman-cap13/my_projects/blob/main/AI_Vs_Human_Text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
files.upload()

In [ ]:

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d shanegerami/ai-vs-human-text

In [ ]:
import zipfile

In [ ]:
with zipfile.ZipFile('/content/ai-vs-human-text.zip','r') as zip_ref:
  zip_ref.extractall()

In [ ]:
import pandas as pd

In [ ]:
df=pd.read_csv('/content/AI_Human.csv')
df

In [ ]:
df = df.sample(20000, random_state=42)

In [ ]:
df['generated'].value_counts()

In [ ]:
# my plan
#1.convert text into token  using AutoTokenizer
#2.split train val dataset using  TensorDataset, DataLoader
#3.make model using AutoModelForClassification
#4 Use loss,optimizer and accelerator
#5 make custom loop
#6. Evaluate it with metrics
#7 Do Error Analysis
#8 return some predictions

In [ ]:
from transformers import AutoTokenizer

tokenizer=AutoTokenizer.from_pretrained('distilbert-base-cased')


In [ ]:
import torch

In [ ]:
def batch_tokenize(texts, labels, batch_size=512):
    input_ids_list = []
    attention_mask_list = []

    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i+batch_size]
        tokenized = tokenizer(
            batch_texts,
            max_length=64,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        input_ids_list.append(tokenized['input_ids'])
        attention_mask_list.append(tokenized['attention_mask'])

    input_ids = torch.cat(input_ids_list, dim=0)
    attention_mask = torch.cat(attention_mask_list, dim=0)
    labels_tensor = torch.tensor(labels, dtype=torch.long)

    return input_ids, attention_mask, labels_tensor


In [ ]:
# def tokenize_text(text,label):
#   token=tokenizer(
#       text,
#       max_length=64,
#       return_tensors=None,
#       truncation=True,
#       padding=True


#   )
#   input_ids=torch.tensor(token['input_ids'])
#   attention_mask=torch.tensor(token['attention_mask'])
#   labels=torch.tensor(label,dtype=torch.long)
#   return input_ids,attention_mask,labels

# train_tokens=tokenize_text(X_train.tolist(),y_train)
# val_tokens=tokenize_text(X_val.tolist(),y_val)


In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_val,y_train,y_val=train_test_split(df['text'],df['generated'], test_size=0.2,random_state=42)

In [ ]:
train_input_ids, train_attention_mask, train_labels = batch_tokenize(X_train.tolist(), y_train.tolist())
val_input_ids, val_attention_mask, val_labels = batch_tokenize(X_val.tolist(), y_val.tolist())


In [ ]:
# train_input_ids, train_attention_mask, train_labels = batch_tokenize(X_train.values.tolist(), y_train.values.tolist())
# val_input_ids, val_attention_mask, val_labels = batch_tokenize(X_val.values.tolist(), y_val.values.tolist())

In [ ]:
#convert train_tokens into train_dataset using DataTensor
from torch.utils.data import TensorDataset

train_dataset=TensorDataset(
    train_input_ids,
    train_attention_mask,
    train_labels

)

val_dataset=TensorDataset(
    val_input_ids,
    val_attention_mask,
    val_labels

)


In [ ]:
#use dataloader
from torch.utils.data import DataLoader

train_loader=DataLoader(
    train_dataset,
    batch_size=64,
    shuffle=True
)

val_loader=DataLoader(
    val_dataset,
    batch_size=64,
    shuffle=True
)

In [ ]:
#make model
from transformers import AutoModelForSequenceClassification

model=AutoModelForSequenceClassification.from_pretrained(
    'distilbert-base-cased',
    num_labels=2
)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)


In [ ]:
#make loss
import torch.nn as nn

loss_fn=nn.CrossEntropyLoss()


In [ ]:
#make optimizer
from torch.optim import Adam

optimizer=Adam(model.parameters(), lr=2e-5)

In [ ]:
# get scheduler
from transformers import get_scheduler

num_epochs=5
num_training_steps=num_epochs*len(train_loader)
lr_scheduler=get_scheduler(
    name='linear',
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)

In [ ]:
#make accelerate
from accelerate import Accelerator

accelerator=Accelerator()
model, optimizer, train_loader, val_loader = accelerator.prepare(
    model, optimizer, train_loader, val_loader
)

In [ ]:
#train the model
from tqdm import tqdm
import numpy as np
from sklearn.metrics import accuracy_score, classification_report
progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
  for batch in train_loader:
    input_ids=batch[0].to(device)
    attention_mask=batch[1].to(device)
    labels=batch[2].to(device)
    outputs=model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
    loss=outputs.loss
    accelerator.backward(loss)

    optimizer.step()
    lr_scheduler.step()
    optimizer.zero_grad()
    progress_bar.update(1)

  model.eval()
  all_preds=[]
  all_labels=[]
  with torch.no_grad():
    for batch in val_loader:
      input_ids=batch[0].to(device)
      attention_mask=batch[1].to(device)
      labels=batch[2].to(device)
      outputs=model(input_ids=input_ids, attention_mask=attention_mask)
      preds=torch.argmax(outputs.logits, dim=1)
      all_preds.extend(preds.cpu().numpy())
      all_labels.extend(labels.cpu().numpy())


    acc = accuracy_score(all_labels, all_preds)
    print(f"Validation Accuracy: {acc:.4f}")
    print("Classification Report:")
    print(classification_report(all_labels, all_preds))

In [ ]:
def predict_text(text):
    model.eval()
    tokens = tokenizer(text, truncation=True, padding=True, max_length=64, return_tensors='pt')
    tokens = {key: val.to(device) for key, val in tokens.items()}

    with torch.no_grad():
        output = model(**tokens)
        logits = output.logits
        prediction = torch.argmax(logits, dim=1).item()

    return "AI-Generated" if prediction == 1 else "Human-Written"


In [ ]:
print(predict_text("In the quiet village, the sun set behind the hills as children laughed."))
print(predict_text("The rapid advancements in quantum algorithms will soon redefine cryptographic landscapes."))


In [ ]:
print(predict_text("All day i am tring to reach the best version of myself"))

In [ ]:
def error_analysis(model, val_loader, tokenizer, device):
    model.eval()
    errors = []

    with torch.no_grad():
        for batch in val_loader:
            input_ids, attention_mask, labels = [x.to(device) for x in batch]
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            preds = torch.argmax(outputs.logits, dim=1)

            for i in range(len(labels)):
                if preds[i] != labels[i]:
                    text_tokens = input_ids[i].cpu()
                    text = tokenizer.decode(text_tokens, skip_special_tokens=True)
                    errors.append({
                        "text": text,
                        "true_label": labels[i].item(),
                        "predicted_label": preds[i].item()
                    })

    return errors


In [ ]:
errors = error_analysis(model, val_loader, tokenizer, device)
print(f"Number of misclassified samples: {len(errors)}")


for i, e in enumerate(errors[:10]):
    print(f"Example {i+1}:")
    print(f"Text: {e['text']}")
    print(f"True Label: {'AI-Generated' if e['true_label'] == 1 else 'Human-Written'}")
    print(f"Predicted Label: {'AI-Generated' if e['predicted_label'] == 1 else 'Human-Written'}")
    print("-" * 50)


In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

def plot_confusion_matrix(model, val_loader, device):
    model.eval()
    all_preds, all_labels = [], []

    with torch.no_grad():
        for batch in val_loader:
            input_ids, attention_mask, labels = [x.to(device) for x in batch]
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            preds = torch.argmax(outputs.logits, dim=1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    cm = confusion_matrix(all_labels, all_preds)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["Human-Written", "AI-Generated"])
    disp.plot(cmap=plt.cm.Blues)
    plt.title("Confusion Matrix")
    plt.show()

plot_confusion_matrix(model, val_loader, device)


In [ ]:
import torch.nn.functional as F

def predict_with_confidence(text):
    model.eval()
    tokens = tokenizer(text, truncation=True, padding=True, max_length=64, return_tensors='pt')
    tokens = {key: val.to(device) for key, val in tokens.items()}

    with torch.no_grad():
        output = model(**tokens)
        logits = output.logits
        probs = F.softmax(logits, dim=1)
        conf, prediction = torch.max(probs, dim=1)

    label = "AI-Generated" if prediction.item() == 1 else "Human-Written"
    return label, conf.item()


text = "This is a sample text to analyze."
label, confidence = predict_with_confidence(text)
print(f"Prediction: {label}, Confidence: {confidence:.2f}")


In [ ]:
train_losses = []
val_accuracies = []
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in train_loader:
        input_ids, attention_mask, labels = batch  # no .to(device) if using accelerator
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

        total_loss += loss.item()

    avg_train_loss = total_loss / len(train_loader)
    train_losses.append(avg_train_loss)

    # Validation
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for batch in val_loader:
            input_ids, attention_mask, labels = batch
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            preds = torch.argmax(outputs.logits, dim=1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    acc = accuracy_score(all_labels, all_preds)
    val_accuracies.append(acc)

    print(f"Epoch {epoch+1}/{num_epochs} — Train Loss: {avg_train_loss:.4f} — Val Accuracy: {acc:.4f}")


In [ ]:
import matplotlib.pyplot as plt

epochs = range(1, num_epochs + 1)

plt.figure(figsize=(10,5))
plt.plot(epochs, train_losses, label='Training Loss')
plt.plot(epochs, val_accuracies, label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Value')
plt.title('Training Loss and Validation Accuracy')
plt.legend()
plt.grid(True)
plt.show()
